In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.2/590.2 MB 325.0 kB/s eta 0:00:00m eta 0:00:010:00:35
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 471.4 kB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 474.5 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 471.5 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 477.6 kB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompa

2024-02-01 16:25:02.614535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 16:25:04.232830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 444.0 kB/s eta 0:00:001m474.8 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 471.6 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 469.5 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 331.2 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 331.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=55349951d63e9143cddf3733649a09fbbce58ccb50e3dd84161b83bbb79c9644
  Stored in directory: /home/benson/.cache/pip/wheels/54/4e/28/3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Un

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-02-01 16:25:50--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   214KB/s    in 1.6s    

2024-02-01 16:25:52 (214 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-02-01 16:25:53--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[=============

In [3]:
import tensorflow as tf
# from tensorflow.keras.layers import StringLookup

# Load data
train_data = pd.read_csv("train-data.tsv", sep='\t')
test_data = pd.read_csv("valid-data.tsv", sep='\t')

train_labels = train_data.iloc[:, 0]
train_features = train_data.iloc[:, 1]

test_labels = test_data.iloc[:, 0]
test_features = test_data.iloc[:, 1]

# Use StringLookup to encode labels
label_lookup = tf.keras.layers.StringLookup(vocabulary=['ham', 'spam'], num_oov_indices=0, mask_token=None)

# Encode labels
train_labels = label_lookup(train_labels)
test_labels = label_lookup(test_labels)

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_len = 100  # Set a maximum length for the sequences
trunc_type = 'post'  # Truncate sequences from the end
oov_token = "<OOV>"  # Out of vocabulary token

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_features)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_features)
test_sequences = tokenizer.texts_to_sequences(test_features)

# Pad sequences to a uniform length
train_padded = pad_sequences(train_sequences, maxlen=max_len, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_len, truncating=trunc_type)

/home/benson/.local/lib/python3.10/site-packages/numpy/core/numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [4]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(36, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_padded,
          train_labels,
          epochs=100,
          validation_data=(test_padded, test_labels))

Epoch 1/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8361 - loss: 0.4519 - val_accuracy: 0.8640 - val_loss: 0.3686
Epoch 2/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8578 - loss: 0.3771 - val_accuracy: 0.8640 - val_loss: 0.3621
Epoch 3/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8664 - loss: 0.3541 - val_accuracy: 0.8640 - val_loss: 0.3464
Epoch 4/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8562 - loss: 0.3489 - val_accuracy: 0.8640 - val_loss: 0.3005
Epoch 5/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8764 - loss: 0.2577 - val_accuracy: 0.9381 - val_loss: 0.1852
Epoch 6/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9495 - loss: 0.1513 - val_accuracy: 0.9723 - val_loss: 0.1196
Epoch 7/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9722 - loss: 0.0951 - val_accuracy: 0.9773 - val_loss: 0.0887
Epoch 8/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9791 - loss: 0.0695 - val_a

In [5]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Tokenize and pad the input text
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_padded = pad_sequences(pred_sequence, maxlen=max_len, truncating=trunc_type)

    # Make predictions
    predictions = model.predict(pred_padded)[0][0]

    # Return a list containing prediction and label
    if predictions >= 0.5:
        return [predictions.astype(int), 'spam']
    else:
        return [predictions.astype(int), 'ham']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
[0, 'ham']


In [6]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(prediction[1])
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
You passed the challenge. Great job!
